In [102]:
import pandas as pd
import plotly.express as px
from datetime import datetime

file_path = 'output.xlsx'
maxpage = 2                                                         # сколько страниц топа грузим
page_size = 60                                                      # число моделей на странице

# Чтение данных
data = pd.read_excel(file_path)

# Дополнительные столбцы
data['Ranking'] = maxpage*page_size - data['N'] + 1
data['Unique_Title'] = data['Link'].apply(lambda x: x.split('/')[-1])
data['Max_Version'] = data['Platform'].apply(lambda x: x.split(' + ')[0] if ' + ' in x else None)
data['File_Type'] = data['Platform'].apply(lambda x: x.split(' + ')[1] if ' + ' in x else None)

data['Pubdate'] = pd.to_datetime(data['Pubdate'])
data['Full_Category'] = data['Category'] + '.' + data['Subcategory']

current_date = datetime.now()
data['Duration'] = (current_date - data['Pubdate']).dt.days

data.drop(['N','Link', 'Standard', 'Pubdate', 'Platform'], axis=1, inplace=True)


In [103]:
display(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2280 entries, 0 to 2279
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Curdate        2280 non-null   object
 1   Title          2280 non-null   object
 2   Category       2280 non-null   object
 3   Subcategory    2280 non-null   object
 4   Corona         2265 non-null   object
 5   V-Ray          1536 non-null   object
 6   Username       2280 non-null   object
 7   Followers      2280 non-null   int64 
 8   Selected       2280 non-null   int64 
 9   Ranking        2280 non-null   int64 
 10  Unique_Title   2280 non-null   object
 11  Max_Version    2272 non-null   object
 12  File_Type      2272 non-null   object
 13  Full_Category  2280 non-null   object
 14  Duration       2280 non-null   int64 
dtypes: int64(4), object(11)
memory usage: 267.3+ KB


None

In [95]:
fig1 = px.line(data, x='Curdate', y='Ranking', color='Unique_Title', title='Top 120 Rank Dynamics')
fig1.show()

c:\Users\mi_al\AppData\Local\Programs\Python\Python39\lib\site-packages\plotly\express\_core.py:1985: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [96]:
data_cat_gr = data.groupby(by='Category', as_index=False)['Ranking'].sum()
data_cat_gr.columns = ['Category', 'Total_Ranking']
data_cat_gr.sort_values(by='Total_Ranking', ascending=False, inplace=True)

fig2 = px.bar(data_cat_gr, x='Category', y='Total_Ranking', title='Top 120 total Rank by Category')
fig2.show()


In [97]:
data_subcat_gr = data.groupby(by=['Full_Category', 'Category'], as_index=False)['Ranking'].sum()
data_subcat_gr.columns = ['Full_Category', 'Category', 'Total_Ranking']
data_subcat_gr.sort_values(by='Total_Ranking', ascending=False, inplace=True)

cat_list = data_subcat_gr['Full_Category'].tolist()
cat_dict = {'Full_Category': cat_list}

fig3 = px.bar(data_subcat_gr, x='Full_Category', y='Total_Ranking', 
              category_orders=cat_dict, color='Category', height=600, title='Top 120 total Rank by Subcategory')
fig3.show()

c:\Users\mi_al\AppData\Local\Programs\Python\Python39\lib\site-packages\plotly\express\_core.py:1985: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [98]:
data_unique_gr = data.groupby(by=['Unique_Title', 'Category', 'Username'], as_index=False)['Ranking'].sum()
data_unique_gr.columns = ['Unique_Title', 'Category', 'Username', 'Total_Ranking']
data_unique_gr.sort_values(by='Total_Ranking', ascending=False, inplace=True)
data_unique_gr = data_unique_gr.head(30)

cat_list = data_unique_gr['Unique_Title'].tolist()
cat_dict = {'Unique_Title': cat_list}

fig4 = px.bar(data_unique_gr, x='Unique_Title', y='Total_Ranking', hover_data='Username',
              category_orders=cat_dict, color='Category', height=800, title='Top 120 total Rank by Unique_Title')
fig4.show()

c:\Users\mi_al\AppData\Local\Programs\Python\Python39\lib\site-packages\plotly\express\_core.py:1985: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [ ]:
data_user_gr = data.groupby(by='Username', as_index=False)['Ranking'].sum()
data_user_gr.columns = ['Username', 'Total_Ranking']
data_user_gr.sort_values(by='Total_Ranking', ascending=False, inplace=True)
data_user_gr = data_user_gr.head(30)

cat_user_list = data_user_gr['Username'].tolist()
cat_user_dict = {'Username': cat_user_list}

fig5 = px.bar(data_user_gr, x='Username', y='Total_Ranking', 
              category_orders=cat_user_dict, height=800, title='Top 120 total Rank by Username')
fig5.show()

In [ ]:
data_max_file = data[['Unique_Title', 'Max_Version', 'File_Type']].drop_duplicates().reset_index()
data_max_file_pivot = data_max_file.pivot_table(
    values='Unique_Title',
    index='Max_Version',
    columns='File_Type',
    aggfunc='count'
)

fig6 = px.imshow(data_max_file_pivot, color_continuous_scale='Blues', 
                 title='Top 120 distribution by Max_Version and File_Type')
fig6.show()

In [109]:
fig1.write_html("Top 120 Rank Dynamics.html")
fig2.write_html("Top 120 total Rank by Category.html")
fig3.write_html("Top 120 total Rank by Subcategory.html")
fig4.write_html("Top 120 total Rank by Unique_Title.html")
fig5.write_html("Top 120 total Rank by Username.html")
fig6.write_html("Top 120 distribution by Max_Version and File_Type.html")

In [79]:
display(data_unique_gr[data_unique_gr['Username']=='Juliet-t-e'])

,Unique_Title,Category,Username,Total_Ranking
254,vezd_na_okhraniaemuiu_territoriiu_parkovku,Architecture,Juliet-t-e,130
